# Python 的 50+ 練習：資料科學學習手冊

> 資料清理轉換專案：中選會選舉資料庫

[數聚點](https://www.datainpoint.com/) | 郭耀仁 <https://linktr.ee/yaojenkuo>

## 專案的第三方模組

<https://github.com/datainpoint/workshop-cec-data-wrangling-with-python/blob/main/requirements.txt>

In [1]:
from string import ascii_uppercase
import os
import re
import pandas as pd
import sqlite3

## 現代資料科學

## 以程式設計做資料科學的應用

![](images/r-for-data-science.png)

來源：[R for Data Science](https://r4ds.had.co.nz)

## 什麼是資料科學的應用場景

- Import 資料的載入。
- **Tidy 資料清理**。
- **Transform 資料外型與類別的轉換**。
- Visualise 探索性分析。
- Model 分析與預測模型。
- Communicate 溝通分享。

## （沒什麼用的冷知識）Wrangle

![](https://media.giphy.com/media/MnlZWRFHR4xruE4N2Z/giphy.gif)

來源：<https://media.giphy.com/media/MnlZWRFHR4xruE4N2Z/giphy.gif>

## 機器學習專案花費 50% 的時間處理 Data Wrangle 的相關任務

![](images/average-time-allocated.png)

來源：<https://www.economist.com/technology-quarterly/2020/06/11/for-ai-data-are-harder-to-come-by-than-you-think>

## 多數的資料清理、資料外型與類別的轉換是面對 `DataFrame`

入門 Pandas 的第一步就是掌握 `Index`、`ndarray`、`Series` 與 `DataFrame` 四個資料結構類別彼此之間的關係。

- `Series` 由 `Index` 與 `ndarray` 組合而成。
- `DataFrame` 由數個共享同一個 `Index` 的 `Series` 組合而成。

## `DataFrame` 是有兩個維度的資料結構

- 第一個維度稱為觀測值（Observations），有時亦稱為列（Rows）
- 第二個維度稱為變數（Variables），有時亦稱為欄（Columns）
- 我們習慣以 `(m, n)` 或者 `m x n` 來描述一個具有 `m` 列觀測值、`n` 欄變數的 `DataFrame`

## `DataFrame` 與二維 `ndarray` 不同的地方

- `DataFrame` 的每個變數可以是異質的。
- `DataFrame` 的觀測值具有列標籤（row-label）、變數具有欄標籤（column-label）

## 什麼是乾淨資料

1. 每個變數有自己的欄位。
2. 每個觀測值有自己的資料列。
3. 每個列、欄標籤與值的對應有自己的儲存格。

![](images/tidy-data.png)

來源：<https://r4ds.had.co.nz/tidy-data.html>

## 不乾淨資料有著各自的樣態

> Tidy datasets are all alike, but every messy dataset is messy in its own way.
>
> Hadley Wickham

來源：<https://r4ds.had.co.nz/tidy-data.html>

## 不乾淨資料

In [2]:
messy_data = pd.DataFrame()
messy_data["column_0"] = ["Luke Skywalker\nAnakinSkywalker", None]
messy_data["column_1"] = ["Luke Skywalker, Mark Hamill", "Anakin Skywalker, Hayden Christensen"]
messy_data

,column_0,column_1
0,Luke Skywalker\nAnakinSkywalker,"Luke Skywalker, Mark Hamill"
1,None,"Anakin Skywalker, Hayden Christensen"


## 乾淨資料

In [3]:
tidy_data = pd.DataFrame()
tidy_data["character_first_name"] = ["Luke", "Anakin"]
tidy_data["character_last_name"] = ["Skywalker", "Skywalker"]
tidy_data["actor_first_name"] = ["Mark", "Hayden"]
tidy_data["actor_last_name"] = ["Hamill", "Christensen"]
tidy_data

,character_first_name,character_last_name,actor_first_name,actor_last_name
0,Luke,Skywalker,Mark,Hamill
1,Anakin,Skywalker,Hayden,Christensen


## 載入資料

## 資料來源為中選會選舉資料庫 www.cec.gov.tw

- [第15任總統(副總統)選舉](https://db.cec.gov.tw/ElecTable/Election/ElecTickets?dataType=tickets&typeId=ELC&subjectId=P0&legisId=00&themeId=1f7d9f4f6bfe06fdaf4db7df2ed4d60c&dataLevel=N&prvCode=00&cityCode=000&areaCode=00&deptCode=000&liCode=0000)
- [第10屆立法委員選舉](https://db.cec.gov.tw/ElecTable/Election/ElecTickets?dataType=areas&typeId=ELC&subjectId=L0&legisId=L1&themeId=be404784efb488c1004009663c892e18&dataLevel=C&prvCode=00&cityCode=000&areaCode=00&deptCode=000&liCode=0000)

## 原始資料格式為試算表

我們可以使用 `pd.read_excel()` 函數載入資料。

## 以臺北市的資料為例

In [4]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
spreadsheet_path = "總統-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
xl = pd.ExcelFile(spreadsheet_path)
print(xl.sheet_names)

['臺北市']


In [5]:
df = pd.read_excel(spreadsheet_path)
df.head()

,第15任總統副總統選舉候選人在臺北市各投開票所得票數一覽表,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,鄉(鎮、市、區)別,村里別,投票所別,各組候選人得票情形,NaN,NaN,有效票數A\nA=1+2+...+N,無效票數B,投票數C\nC=A+B,已領未投票數\nD\nD=E-C,發出票數E\nE=C+D,用餘票數F,選舉人數G\nG=E+F,投票率H\nH=C÷G
1,NaN,NaN,NaN,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,總 計,NaN,NaN,"70,769","685,830","875,854","1,632,453","21,381","1,653,834",143,"1,653,977","513,287","2,167,264",76.3098


## 載入臺北市的區域立委資料

- 區域立委的試算表比較複雜，因為每個縣市中可能有多個選區。
- 載入前先檢查活頁簿中的試算表清單。

In [6]:
file_name = "區域立委-A05-2-得票數一覽表(臺北市).xls"
spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
xl = pd.ExcelFile(spreadsheet_path)
print(xl.sheet_names)

['臺北市第1選舉區', '臺北市第2選舉區', '臺北市第3選舉區', '臺北市第4選舉區', '臺北市第5選舉區', '臺北市第6選舉區', '臺北市第7選舉區', '臺北市第8選舉區']


## 透過 `sheet_name` 參數指定要載入活頁簿中的哪一個試算表

In [7]:
regional_legislator_taipei_city = {}
for sheet_name in xl.sheet_names:
    regional_legislator_taipei_city[sheet_name] = pd.read_excel(spreadsheet_path, sheet_name=sheet_name)
for k, v in regional_legislator_taipei_city.items():
    print("Shape of {} : {}".format(k, v.shape))

Shape of 臺北市第1選舉區 : (219, 18)
Shape of 臺北市第2選舉區 : (224, 17)
Shape of 臺北市第3選舉區 : (233, 15)
Shape of 臺北市第4選舉區 : (248, 18)
Shape of 臺北市第5選舉區 : (225, 19)
Shape of 臺北市第6選舉區 : (200, 21)
Shape of 臺北市第7選舉區 : (222, 16)
Shape of 臺北市第8選舉區 : (212, 20)


In [8]:
file_name = "不分區立委-A05-6-得票數一覽表(臺北市).xls"
spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
xl = pd.ExcelFile(spreadsheet_path)
print(xl.sheet_names)

['臺北市']


In [9]:
file_name = "山地立委-A05-4-得票數一覽表(臺北市).xls"
spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
xl = pd.ExcelFile(spreadsheet_path)
print(xl.sheet_names)

['臺北市']


In [10]:
file_name = "平地立委-A05-4-得票數一覽表(臺北市).xls"
spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
xl = pd.ExcelFile(spreadsheet_path)
print(xl.sheet_names)

['臺北市']


## 資料清理

## 哪些因素使得原本的試算表不是「乾淨資料」？

- 合併儲存格。
- 未定義值、遺漏值。
- 在觀測值中參雜了「小計」與「總計」。
- 資料值（候選人、政黨）記錄在變數名稱中。

## 載入試算表時使用 `skiprows` 參數略過合併儲存格

In [11]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
spreadsheet_path = "總統-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
df = pd.read_excel(spreadsheet_path, skiprows=[0, 1, 3, 4], thousands=',')

In [12]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,總 計,NaN,NaN,70769,685830,875854,1632453,21381,1653834,143,1653977,513287,2167264,76.3098
1,松山區,NaN,NaN,5436,55918,64207,125561,1762,127323,2,127325,37329,164654,77.3276
2,NaN,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,NaN,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,NaN,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811


## 更新資料框的 `columns` 屬性

In [13]:
n_cols = df.columns.size
n_candidates = n_cols - 11
id_vars = ['town', 'village', 'office']
candidates = list(df.columns[3:(3 + n_candidates)])
office_cols = list(ascii_uppercase[:8])
col_names = id_vars + candidates + office_cols
df.columns = col_names

In [14]:
print(n_candidates)
print(candidates)
print(office_cols)
print(col_names)

3
['(1)\n宋楚瑜\n余湘', '(2)\n韓國瑜\n張善政', '(3)\n蔡英文\n賴清德']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
['town', 'village', 'office', '(1)\n宋楚瑜\n余湘', '(2)\n韓國瑜\n張善政', '(3)\n蔡英文\n賴清德', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']


## 使用 `df.fillna()` 方法前向填補 `town` 欄位中的未定義值

`ffill` 參數：利用前一個有效值填補未定義值，直到下一個有效值。

In [15]:
filled_towns = df['town'].fillna(method='ffill')
df = df.assign(town=filled_towns)
df.head()

,town,village,office,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,A,B,C,D,E,F,G,H
0,總 計,NaN,NaN,70769,685830,875854,1632453,21381,1653834,143,1653977,513287,2167264,76.3098
1,松山區,NaN,NaN,5436,55918,64207,125561,1762,127323,2,127325,37329,164654,77.3276
2,松山區,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,松山區,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,松山區,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811


## 使用 `df.dropna()` 移除「小計」與「總計」

- 遵循「乾淨資料」法則。
- 避免錯誤的加總。

In [16]:
df = df.dropna()
df.head()

,town,village,office,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,A,B,C,D,E,F,G,H
2,松山區,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,松山區,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,松山區,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811
5,松山區,莊敬里,576.0,43,389,462,894,14,908,0,908,271,1179,77.0144
6,松山區,東榮里,577.0,38,431,545,1014,18,1032,0,1032,272,1304,79.1411


## 使用 `str.replace()` 方法取代多餘的特殊文字 `"\u3000"`

In [17]:
print(df['town'].unique())
stripped_strict = df['town'].str.replace("\u3000", "")
df = df.assign(town=stripped_strict)
print(df['town'].unique())

['\u3000松山區' '\u3000信義區' '\u3000大安區' '\u3000中山區' '\u3000中正區' '\u3000大同區'
 '\u3000萬華區' '\u3000文山區' '\u3000南港區' '\u3000內湖區' '\u3000士林區' '\u3000北投區']
['松山區' '信義區' '大安區' '中山區' '中正區' '大同區' '萬華區' '文山區' '南港區' '內湖區' '士林區' '北投區']


## 使用 `pd.melt()` 函數轉置資料框

In [18]:
df = df.drop(labels=office_cols, axis=1)
df_long = pd.melt(df,
                  id_vars=id_vars,
                  var_name='candidate_info',
                  value_name='votes'
                 )
df_long.head()

,town,village,office,candidate_info,votes
0,松山區,莊敬里,573.0,(1)\n宋楚瑜\n余湘,36
1,松山區,莊敬里,574.0,(1)\n宋楚瑜\n余湘,46
2,松山區,莊敬里,575.0,(1)\n宋楚瑜\n余湘,48
3,松山區,莊敬里,576.0,(1)\n宋楚瑜\n余湘,43
4,松山區,東榮里,577.0,(1)\n宋楚瑜\n余湘,38


## 定義函數 `tidy_dataframe()` 將前述的資料操作組織起來

In [19]:
def tidy_dataframe(df):
    # updating columns attributes 
    n_cols = df.columns.size
    n_candidates = n_cols - 11
    id_vars = ['town', 'village', 'office']
    candidates = list(df.columns[3:(3 + n_candidates)])
    office_cols = list(ascii_uppercase[:8])
    col_names = id_vars + candidates + office_cols
    df.columns = col_names
    # forward-fill district values
    filled_towns = df['town'].fillna(method='ffill')
    df = df.assign(town=filled_towns)
    # removing summations
    df = df.dropna()
    # removing extra spaces
    stripped_towns = df['town'].str.replace("\u3000", "")
    df = df.assign(town=stripped_towns)
    # pivoting
    df = df.drop(labels=office_cols, axis=1)
    tidy_df = pd.melt(df,
                      id_vars=id_vars,
                      var_name='candidate_info',
                      value_name='votes'
                     )
    return tidy_df

## 將縣市名稱從檔名中取出

In [20]:
files = [f for f in os.listdir("總統-各投票所得票明細及概況(Excel檔)/") if not f.startswith('.') and "A05-4" in f] # to skip those hidden files
counties = [re.split("\(|\)", f)[1] for f in files]
print(counties)

['宜蘭縣', '彰化縣', '金門縣', '桃園市', '苗栗縣', '臺南市', '雲林縣', '南投縣', '高雄市', '臺北市', '新北市', '花蓮縣', '新竹市', '新竹縣', '基隆市', '連江縣', '嘉義縣', '嘉義市', '屏東縣', '澎湖縣', '臺東縣', '臺中市']


## 應用 `tidy_dataframe()` 函數

In [21]:
presidential = pd.DataFrame()
for county in counties:
    file_name = "總統-A05-4-候選人得票數一覽表-各投開票所({}).xls".format(county)
    spreadsheet_path = "總統-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_path, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['county'] = county
    presidential = pd.concat([presidential, tidy_df])
    print("Tidying {}".format(file_name))
presidential = presidential.reset_index(drop=True) # reset index for the appended dataframe

Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(宜蘭縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(彰化縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(金門縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(桃園市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(苗栗縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺南市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(雲林縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(南投縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(高雄市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(花蓮縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(基隆市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(連江縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(屏東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(澎湖縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺中市).xls


In [22]:
presidential.head()

,town,village,office,candidate_info,votes,county
0,宜蘭市,民族里,1.0,(1)\n宋楚瑜\n余湘,37,宜蘭縣
1,宜蘭市,民族里,2.0,(1)\n宋楚瑜\n余湘,31,宜蘭縣
2,宜蘭市,建軍里,3.0,(1)\n宋楚瑜\n余湘,19,宜蘭縣
3,宜蘭市,建軍里,4.0,(1)\n宋楚瑜\n余湘,29,宜蘭縣
4,宜蘭市,泰山里,5.0,(1)\n宋楚瑜\n余湘,25,宜蘭縣


In [23]:
presidential.tail()

,town,village,office,candidate_info,votes,county
51673,和平區,梨山里,1845.0,(3)\n蔡英文\n賴清德,132,臺中市
51674,和平區,梨山里,1846.0,(3)\n蔡英文\n賴清德,107,臺中市
51675,和平區,梨山里,1847.0,(3)\n蔡英文\n賴清德,40,臺中市
51676,和平區,平等里,1848.0,(3)\n蔡英文\n賴清德,24,臺中市
51677,和平區,平等里,1849.0,(3)\n蔡英文\n賴清德,102,臺中市


## 定義函數 `adjust_presidential()` 調整

In [24]:
def adjust_presidential(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "", regex=False)
    candidates = candidate_info_df[1].str.cat(candidate_info_df[2], sep="/")
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['office'] = df['office'].astype(int)
    df = df[['county', 'town', 'village', 'office', 'number', 'candidate', 'votes']]
    return df

## 應用 `adjust_presidential()` 函數

In [25]:
presidential = adjust_presidential(presidential)

In [26]:
presidential.head()

,county,town,village,office,number,candidate,votes
0,宜蘭縣,宜蘭市,民族里,1,(1),宋楚瑜/余湘,37
1,宜蘭縣,宜蘭市,民族里,2,(1),宋楚瑜/余湘,31
2,宜蘭縣,宜蘭市,建軍里,3,(1),宋楚瑜/余湘,19
3,宜蘭縣,宜蘭市,建軍里,4,(1),宋楚瑜/余湘,29
4,宜蘭縣,宜蘭市,泰山里,5,(1),宋楚瑜/余湘,25


In [27]:
presidential.tail()

,county,town,village,office,number,candidate,votes
51673,臺中市,和平區,梨山里,1845,(3),蔡英文/賴清德,132
51674,臺中市,和平區,梨山里,1846,(3),蔡英文/賴清德,107
51675,臺中市,和平區,梨山里,1847,(3),蔡英文/賴清德,40
51676,臺中市,和平區,平等里,1848,(3),蔡英文/賴清德,24
51677,臺中市,和平區,平等里,1849,(3),蔡英文/賴清德,102


## 完成了`總統-各投票所得票明細及概況`！

![](https://media.giphy.com/media/1sjwSoZLcENCE/giphy.gif)

來源：<https://media.giphy.com/media/1sjwSoZLcENCE/giphy.gif>

## 應用 `tidy_dataframe()` 函數在`立委-各投票所得票明細及概況`

三種立委類型：Types of legislator:

1. 區域立委。
2. 平地原住民立委、山地原住民立委。
3. 不分區立委。

區域立委、平地原住民立委、山地原住民立委是投給「候選人」，不分區立委是投給「政黨」。

## 應用 `tidy_dataframe()` 函數在區域立委試算表

In [28]:
regional = pd.DataFrame()
for county in counties:
    file_name = "區域立委-A05-2-得票數一覽表({}).xls".format(county)
    spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
    xl = pd.ExcelFile(spreadsheet_path)
    for sheet in xl.sheet_names:
        # skip those combined cells
        df = pd.read_excel(spreadsheet_path, skiprows=[0, 1, 3, 4], thousands=',', sheet_name=sheet)
        tidy_df = tidy_dataframe(df)
        # appending dataframe of each city/county
        tidy_df['county'] = county
        tidy_df['electoral_district'] = sheet
        regional = pd.concat([regional, tidy_df])
        print("Tidying {} of {}".format(sheet, file_name))
regional = regional.reset_index(drop=True) # reset index for the appended dataframe

Tidying 宜蘭縣選舉區 of 區域立委-A05-2-得票數一覽表(宜蘭縣).xls
Tidying 彰化縣第1選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第2選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第3選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第4選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 金門縣選舉區 of 區域立委-A05-2-得票數一覽表(金門縣).xls
Tidying 桃園市第1選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第2選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第3選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第4選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第5選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第6選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 苗栗縣第1選舉區 of 區域立委-A05-2-得票數一覽表(苗栗縣).xls
Tidying 苗栗縣第2選舉區 of 區域立委-A05-2-得票數一覽表(苗栗縣).xls
Tidying 臺南市第1選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第2選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第3選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第4選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第5選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第6選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 雲林縣第1選舉區 of 區域立委-A05-2-得票數一覽表(雲林縣).xls
Tidying 雲林縣第2選舉區 

In [29]:
regional.head()

,town,village,office,candidate_info,votes,county,electoral_district
0,宜蘭市,民族里,1.0,(1)\n黃定和\n無,128,宜蘭縣,宜蘭縣選舉區
1,宜蘭市,民族里,2.0,(1)\n黃定和\n無,138,宜蘭縣,宜蘭縣選舉區
2,宜蘭市,建軍里,3.0,(1)\n黃定和\n無,122,宜蘭縣,宜蘭縣選舉區
3,宜蘭市,建軍里,4.0,(1)\n黃定和\n無,104,宜蘭縣,宜蘭縣選舉區
4,宜蘭市,泰山里,5.0,(1)\n黃定和\n無,129,宜蘭縣,宜蘭縣選舉區


## 定義函數 `adjust_legislative()` 調整

In [30]:
def adjust_legislative(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "", regex=False)
    candidates = candidate_info_df[1]
    parties = candidate_info_df[2]
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['party'] = parties
    df['office'] = df['office'].astype(int)
    return df

## 應用 `adjust_legislative()` 函數

In [31]:
regional = adjust_legislative(regional)
regional = regional[['county', 'electoral_district', 'town', 'village', 'office', 'number', 'party', 'candidate', 'votes']]

In [32]:
regional.head()

,county,electoral_district,town,village,office,number,party,candidate,votes
0,宜蘭縣,宜蘭縣選舉區,宜蘭市,民族里,1,(1),無,黃定和,128
1,宜蘭縣,宜蘭縣選舉區,宜蘭市,民族里,2,(1),無,黃定和,138
2,宜蘭縣,宜蘭縣選舉區,宜蘭市,建軍里,3,(1),無,黃定和,122
3,宜蘭縣,宜蘭縣選舉區,宜蘭市,建軍里,4,(1),無,黃定和,104
4,宜蘭縣,宜蘭縣選舉區,宜蘭市,泰山里,5,(1),無,黃定和,129


## 應用 `tidy_dataframe()` 函數在平地原住民立委、山地原住民立委試算表

In [33]:
indigenous = pd.DataFrame()
indigenous_types = ['山地', '平地']
for county in counties:
    for indigenous_type in indigenous_types:
        file_name = "{}立委-A05-4-得票數一覽表({}).xls".format(indigenous_type, county)
        spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
        # skip those combined cells
        df = pd.read_excel(spreadsheet_path, skiprows=[0, 1, 3, 4], thousands=',')
        tidy_df = tidy_dataframe(df)
        # appending dataframe of each city/county
        tidy_df['county'] = county
        tidy_df['electoral_district'] = '{}原住民'.format(indigenous_type)
        indigenous = pd.concat([indigenous, tidy_df])
        print("Tidying {}".format(file_name))
indigenous = indigenous.reset_index(drop=True) # reset index for the appended dataframe

Tidying 山地立委-A05-4-得票數一覽表(宜蘭縣).xls
Tidying 平地立委-A05-4-得票數一覽表(宜蘭縣).xls
Tidying 山地立委-A05-4-得票數一覽表(彰化縣).xls
Tidying 平地立委-A05-4-得票數一覽表(彰化縣).xls
Tidying 山地立委-A05-4-得票數一覽表(金門縣).xls
Tidying 平地立委-A05-4-得票數一覽表(金門縣).xls
Tidying 山地立委-A05-4-得票數一覽表(桃園市).xls
Tidying 平地立委-A05-4-得票數一覽表(桃園市).xls
Tidying 山地立委-A05-4-得票數一覽表(苗栗縣).xls
Tidying 平地立委-A05-4-得票數一覽表(苗栗縣).xls
Tidying 山地立委-A05-4-得票數一覽表(臺南市).xls
Tidying 平地立委-A05-4-得票數一覽表(臺南市).xls
Tidying 山地立委-A05-4-得票數一覽表(雲林縣).xls
Tidying 平地立委-A05-4-得票數一覽表(雲林縣).xls
Tidying 山地立委-A05-4-得票數一覽表(南投縣).xls
Tidying 平地立委-A05-4-得票數一覽表(南投縣).xls
Tidying 山地立委-A05-4-得票數一覽表(高雄市).xls
Tidying 平地立委-A05-4-得票數一覽表(高雄市).xls
Tidying 山地立委-A05-4-得票數一覽表(臺北市).xls
Tidying 平地立委-A05-4-得票數一覽表(臺北市).xls
Tidying 山地立委-A05-4-得票數一覽表(新北市).xls
Tidying 平地立委-A05-4-得票數一覽表(新北市).xls
Tidying 山地立委-A05-4-得票數一覽表(花蓮縣).xls
Tidying 平地立委-A05-4-得票數一覽表(花蓮縣).xls
Tidying 山地立委-A05-4-得票數一覽表(新竹市).xls
Tidying 平地立委-A05-4-得票數一覽表(新竹市).xls
Tidying 山地立委-A05-4-得票數一覽表(新竹縣).xls
Tidying 平地立委-A05-4-得票數一覽表(新竹縣).xls
Tidying 山地立委-A05-4-得

In [34]:
indigenous.head()

,town,village,office,candidate_info,votes,county,electoral_district
0,宜蘭市,民族里,1.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
1,宜蘭市,民族里,2.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
2,宜蘭市,建軍里,3.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
3,宜蘭市,建軍里,4.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
4,宜蘭市,泰山里,5.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民


## 應用 `adjust_legislative()` 函數

In [35]:
indigenous = adjust_legislative(indigenous)
indigenous = indigenous[['county', 'electoral_district', 'town', 'village', 'office', 'number', 'party', 'candidate', 'votes']]

In [36]:
indigenous.head()

,county,electoral_district,town,village,office,number,party,candidate,votes
0,宜蘭縣,山地原住民,宜蘭市,民族里,1,(1),民主進步黨,伍麗華Saidhai．Tahovecahe,0
1,宜蘭縣,山地原住民,宜蘭市,民族里,2,(1),民主進步黨,伍麗華Saidhai．Tahovecahe,0
2,宜蘭縣,山地原住民,宜蘭市,建軍里,3,(1),民主進步黨,伍麗華Saidhai．Tahovecahe,0
3,宜蘭縣,山地原住民,宜蘭市,建軍里,4,(1),民主進步黨,伍麗華Saidhai．Tahovecahe,0
4,宜蘭縣,山地原住民,宜蘭市,泰山里,5,(1),民主進步黨,伍麗華Saidhai．Tahovecahe,0


## 區域立委、平地原住民立委與山地原住民立委是投給「候選人」

In [37]:
legislative_regional = pd.concat([regional, indigenous], axis=0)
legislative_regional = legislative_regional.reset_index(drop=True)

## 應用 `tidy_dataframe()` 函數在不分區立委試算表

In [38]:
legislative_at_large = pd.DataFrame()
for county in counties:
    file_name = "不分區立委-A05-6-得票數一覽表({}).xls".format(county)
    spreadsheet_path = "立委-各投票所得票明細及概況(Excel檔)/{}".format(file_name)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_path, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['county'] = county
    legislative_at_large = pd.concat([legislative_at_large,tidy_df])
    print("Tidying {}".format(file_name))
legislative_at_large = legislative_at_large.reset_index(drop=True) # reset index for the appended dataframe

Tidying 不分區立委-A05-6-得票數一覽表(宜蘭縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(彰化縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(金門縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(桃園市).xls
Tidying 不分區立委-A05-6-得票數一覽表(苗栗縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺南市).xls
Tidying 不分區立委-A05-6-得票數一覽表(雲林縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(南投縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(高雄市).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺北市).xls
Tidying 不分區立委-A05-6-得票數一覽表(新北市).xls
Tidying 不分區立委-A05-6-得票數一覽表(花蓮縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(新竹市).xls
Tidying 不分區立委-A05-6-得票數一覽表(新竹縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(基隆市).xls
Tidying 不分區立委-A05-6-得票數一覽表(連江縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(嘉義縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(嘉義市).xls
Tidying 不分區立委-A05-6-得票數一覽表(屏東縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(澎湖縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺東縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺中市).xls


In [39]:
legislative_at_large.head()

,town,village,office,candidate_info,votes,county
0,宜蘭市,民族里,1.0,(1)\n\n合一行動聯盟,2,宜蘭縣
1,宜蘭市,民族里,2.0,(1)\n\n合一行動聯盟,1,宜蘭縣
2,宜蘭市,建軍里,3.0,(1)\n\n合一行動聯盟,2,宜蘭縣
3,宜蘭市,建軍里,4.0,(1)\n\n合一行動聯盟,0,宜蘭縣
4,宜蘭市,泰山里,5.0,(1)\n\n合一行動聯盟,3,宜蘭縣


## 應用 `adjust_legislative()` 函數調整

In [40]:
legislative_at_large = adjust_legislative(legislative_at_large)
legislative_at_large = legislative_at_large[['county', 'town', 'village', 'office', 'number', 'party', 'votes']]
legislative_at_large.head()

,county,town,village,office,number,party,votes
0,宜蘭縣,宜蘭市,民族里,1,(1),合一行動聯盟,2
1,宜蘭縣,宜蘭市,民族里,2,(1),合一行動聯盟,1
2,宜蘭縣,宜蘭市,建軍里,3,(1),合一行動聯盟,2
3,宜蘭縣,宜蘭市,建軍里,4,(1),合一行動聯盟,0
4,宜蘭縣,宜蘭市,泰山里,5,(1),合一行動聯盟,3


## 我們可以定義類別 `TaiwanElection2020` 更進一步把前述程式封裝起來

<https://github.com/datainpoint/workshop-cec-data-wrangling-with-python/blob/main/taiwan_election_2020.py>

In [41]:
from taiwan_election_2020 import TaiwanElection2020

tw_election = TaiwanElection2020()
presidential = tw_election.generate_presidential()
legislative_regional = tw_election.generate_legislative_regional()
legislative_at_large = tw_election.generate_legislative_at_large()

Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(宜蘭縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(彰化縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(金門縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(桃園市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(苗栗縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺南市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(雲林縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(南投縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(高雄市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(花蓮縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(基隆市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(連江縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(屏東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(澎湖縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺中市).xls
Tidying 宜蘭縣選舉區 of 區域立委-A05-2-得票數一覽表(宜蘭縣).xls
Tidying 彰化縣第1選舉區 of 區域立委-A05-2-

## 完成了`立委-各投票所得票明細及概況`！

![](https://media.giphy.com/media/13tw3BAKGrRMRy/giphy.gif)

來源：<https://media.giphy.com/media/13tw3BAKGrRMRy/giphy.gif>

## 使用 `df.to_csv()` 方法將乾淨資料匯出成為 CSV 檔案

我們通常指定參數 `index=False` 因為檔案中不需要 `Index`

In [42]:
presidential.to_csv('presidential.csv', index=False)
legislative_regional.to_csv('legislative_regional.csv', index=False)
legislative_at_large.to_csv('legislative_at_large.csv', index=False)

## 使用 `df.to_sql()` 方法將乾淨資料會出成為 SQLite 資料庫檔案

In [43]:
conn = sqlite3.connect('tw-election-2020.db')
presidential.to_sql('presidential', con=conn, index=False, if_exists='replace')
legislative_regional.to_sql('legislative_regional', con=conn, index=False, if_exists='replace')
legislative_at_large.to_sql('legislative_at_large', con=conn, index=False, if_exists='replace')

327294